# ECDH and ECDSA

**Module 06** | 06-elliptic-curves

*Key exchange and signatures on curves*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **ecdh and ecdsa**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Completion of [Scalar Multiplication](06e-scalar-multiplication.ipynb).
- Concepts and notation introduced in the previous notebook.

## ECDH Key Exchange

In [ ]:
# Same Diffie-Hellman idea, but on an elliptic curve
E = EllipticCurve(GF(next_prime(10^6)), [1, 1])
P = E.random_point()
n = P.order()
a = randint(1, n-1)  # Alice's secret
b = randint(1, n-1)  # Bob's secret
A = a * P  # Alice's public
B = b * P  # Bob's public
print(f'Shared secret match: {a*B == b*A}')

## ECDSA Signing

In [ ]:
# Sign a message hash with ECDSA
# TODO: implement ECDSA signing step by step

## ECDSA Verification

In [ ]:
# Verify an ECDSA signature
# TODO: implement ECDSA verification step by step

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **ecdh and ecdsa**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*